In [1]:
import torch 
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np

/opt/anaconda3/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # ResNet expects 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Define dataset paths
train_dir = "/Users/emmanuelgeorgep/Desktop/temp/Dataset/Train"
test_dir = "/Users/emmanuelgeorgep/Desktop/temp/Dataset/Test"

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print dataset info
print("Train samples:", len(train_dataset))
print("Test samples:", len(test_dataset))
print("Classes:", train_dataset.classes)

Train samples: 12000
Test samples: 2000
Classes: ['Blocked', 'Normal']


In [3]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes=2, f=32, k=3, u=256, r=0.5):
        super(CustomCNN, self).__init__()

        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=f, kernel_size=k, padding=1),
            nn.BatchNorm2d(f),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels=f, out_channels=f, kernel_size=k, padding=1),
            nn.BatchNorm2d(f),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels=f, out_channels=f, kernel_size=k, padding=1),
            nn.BatchNorm2d(f),
            nn.MaxPool2d(kernel_size=2),
        )

        self.flatten = nn.Flatten()

        self.classifier = nn.Sequential(
            nn.Linear(f * 16 * 16, u),  # assuming input image size is 128x128
            nn.ReLU(),
            nn.BatchNorm1d(u),
            nn.Dropout(r),
            nn.Linear(u, num_classes)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

In [4]:
from torchsummary import summary
import torch

model = CustomCNN()
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             896
       BatchNorm2d-2         [-1, 32, 128, 128]              64
         MaxPool2d-3           [-1, 32, 64, 64]               0
            Conv2d-4           [-1, 32, 64, 64]           9,248
       BatchNorm2d-5           [-1, 32, 64, 64]              64
         MaxPool2d-6           [-1, 32, 32, 32]               0
            Conv2d-7           [-1, 32, 32, 32]           9,248
       BatchNorm2d-8           [-1, 32, 32, 32]              64
         MaxPool2d-9           [-1, 32, 16, 16]               0
          Flatten-10                 [-1, 8192]               0
           Linear-11                  [-1, 256]       2,097,408
             ReLU-12                  [-1, 256]               0
      BatchNorm1d-13                  [-1, 256]             512
          Dropout-14                  [

In [5]:
def train_model(model, train_loader, criterion, optimizer, device, epochs=5):
    model.to('mps')
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        losses.append(running_loss/len(train_loader))
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

def test_model(model, test_loader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

In [6]:
model = CustomCNN(num_classes=len(train_dataset.classes)).to('mps')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
losses = []

train_model(model, train_loader, criterion, optimizer, "mps", epochs=5)
test_model(model, test_loader, "mps", class_names=train_dataset.classes)

Epoch [1/5], Loss: 0.2358
Epoch [2/5], Loss: 0.1327
Epoch [3/5], Loss: 0.0983
Epoch [4/5], Loss: 0.0831
Epoch [5/5], Loss: 0.0793

Classification Report:
              precision    recall  f1-score   support

     Blocked       0.97      0.95      0.96      1000
      Normal       0.95      0.97      0.96      1000

    accuracy                           0.96      2000
   macro avg       0.96      0.96      0.96      2000
weighted avg       0.96      0.96      0.96      2000



In [8]:
torch.save(model.state_dict(), "Student-KL.pth")